In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import sys
from decimal import Decimal
sys.path.append("/home/lvotapka/Downloads/mmvt_seekr")
import mmvt_seekr as seekr
import numpy as np
#import numpy as np

In [2]:
bound_states = [0]
verbose=False
conv_stride = 100000

# Calculate kinetics with full data

In [3]:
model, max_steps = seekr.model.make_model(milestone_filename="milestones.xml")
p_equil, N, R, T, T_tot, Q, n_conv, r_conv, k_cell= seekr.analyze.analyze_kinetics(model, bound_states, verbose=True,)
MFPT = T[0]
k_off = 1/MFPT


FileNotFoundError: [Errno 2] No such file or directory: 'milestones.xml'

In [ ]:
k_on = seekr.analyze.calc_kon_from_bd(model, bound_states, Q)


In [ ]:
print("MLE k_on", '%.2E' % Decimal(k_on), "M^-1 s^-1")
print("MLE k_off", '%.2E' % Decimal(k_off), "s^-1")


In [ ]:
num = 500
skip = 100
stride = 100


In [ ]:
k_off_list, running_avg, running_std, k_on_list, k_on_avg_list, k_on_std_list = seekr.analyze.monte_carlo_milestoning_error(
    model, bound_states, Q, N, R, p_equil ,T_tot, num=num, skip =skip,stride = stride, verbose=False)
k_off_std = np.std(k_off_list)
k_on_std = np.std(k_on_list)



In [ ]:
print("k_off entries:", len(k_off_list))
print("avg k off", '%.2E' % Decimal(np.average(k_off_list))," +- ", '%.2E' % Decimal(k_off_std), " s^-1") 
print("avg k on", '%.2E' % Decimal(np.average(k_on_list))," +- ", '%.2E' % Decimal(k_on_std), " M^-1 s^-1")

In [ ]:
seekr.plots.MCMC_conv(running_avg, running_std)
seekr.plots.MCMC_conv(k_on_avg_list, k_on_std_list)


# Milestone Convergence

In [ ]:
conv_stride = 500000 #1 ns stride
conv_skip = 5000000 #skip first 10 ns

In [ ]:
bound_dict = [0]
N_conv, R_conv, k_cell_conv, p_equil_conv, k_conv, k_on_conv, conv_intervals = seekr.analyze.check_milestone_convergence(
    model, bound_states, conv_stride, conv_skip, max_steps,)

n_fig, ax = seekr.plots.plot_n_conv(N_conv, conv_intervals)
r_fig = seekr.plots.plot_r_conv(R_conv, conv_intervals)
p_fig, ax = seekr.plots.plot_p_equil(p_equil_conv, conv_intervals)
k_fig, ax = seekr.plots.plot_k_off_conv(k_conv, conv_intervals)

# Determine Minimum Simulation times from RMSD windows

In [ ]:
print(len(conv_intervals))

In [ ]:
window = 30 # 30 ns
cutoff = 0.05 #5%
conv_windows = 20 # 50 ns

In [ ]:

seekr.plots.plot_window_rmsd(N_conv, conv_intervals, window)
seekr.plots.plot_window_rmsd(R_conv, conv_intervals, window)
min_anchor_times = seekr.analyze.calc_RMSD_conv(model, N_conv, R_conv, conv_intervals, window, cutoff, conv_windows)
print(min_anchor_times)

In [ ]:
p_equil, N, R, T, T_tot, Q, n_conv, r_conv, k_cell= seekr.analyze.analyze_kinetics(
     model, bound_states, max_steps = min_anchor_times, verbose=True,)
k_off_list, running_avg, running_std,k_on_list, k_on_avg_list, k_on_std_list = seekr.analyze.monte_carlo_milestoning_error(
    model, bound_states, Q, N, R, p_equil,T_tot, num = 500, skip =100,stride = 100, verbose=False)
#seekr.plots.MCMC_conv(running_avg, running_std)

In [ ]:
MFPT = T[0]
k_off = 1/MFPT
print("MLE k_off", '%.2E' % Decimal(k_off))
k_off_std = np.std(k_off_list)
k_on_std = np.std(k_on_list)
print("k_off entries:", len(k_off_list))
print("avg k off", '%.2E' % Decimal(np.average(k_off_list))," +- ", '%.2E' % Decimal(k_off_std), " s^-1") 
print("avg k on", '%.2E' % Decimal(np.average(k_on_list))," +- ", '%.2E' % Decimal(k_on_std), " M^-1 s^-1")

In [ ]:
window = 30 # 30 ns
cutoff = 0.01 #1%
conv_windows = 20 # 20 ns

In [ ]:
seekr.plots.plot_window_rmsd(N_conv, conv_intervals, window)
seekr.plots.plot_window_rmsd(R_conv, conv_intervals, window)
min_anchor_times = seekr.analyze.calc_RMSD_conv(model, N_conv, R_conv, conv_intervals, window, cutoff, conv_windows)
print(min_anchor_times)

In [ ]:
num = 500
skip = 100
stride = 100

In [ ]:
print()

In [ ]:
p_equil, N, R, T, T_tot, Q, n_conv, r_conv, k_cell= seekr.analyze.analyze_kinetics(
     model, bound_dict, max_steps = min_anchor_times, verbose=True,)
k_off_list, running_avg, running_std,k_on_list, k_on_avg_list, k_on_std_list = seekr.analyze.monte_carlo_milestoning_error(
    model, bound_dict, Q, N, R, p_equil,T_tot, num = num, skip =skip,stride = stride, verbose=False)
#seekr.plots.MCMC_conv(running_avg, running_std)

In [ ]:
MFPT = T[0]
k_off = 1/MFPT
print("MLE k_off", '%.2E' % Decimal(k_off))
k_off_std = np.std(k_off_list)
k_on_std = np.std(k_on_list)
print("k_off entries:", len(k_off_list))
print("avg k off", '%.2E' % Decimal(np.average(k_off_list))," +- ", '%.2E' % Decimal(k_off_std), " s^-1") 
print("avg k on", '%.2E' % Decimal(np.average(k_on_list))," +- ", '%.2E' % Decimal(k_on_std), " M^-1 s^-1")